In [1]:
import fasttext
import numpy as np
import json
import random
import datetime
import os

Via https://fasttext.cc

```
wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.it.300.bin.gz
wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.it.zip
```

In [2]:
wikimodel = fasttext.FastText.load_model("data/wiki.it/wiki.it.bin")

ccmodel = fasttext.FastText.load_model("data/cc.it.300.bin")

In [3]:
def cosine_distance(first, second, model, decimals=2):
    exact_distance = (np.dot(model[first], model[second])
            /(np.linalg.norm(model[first])*np.linalg.norm(model[second]))
           )
    truncated_distance = np.round(float(exact_distance*100),decimals)
    return truncated_distance

## Extract Apple dictionary

https://gist.github.com/josephg/5e134adf70760ee7e49d

In [16]:
# Find Italian dictionary

for entry in os.scandir("/System/Library/AssetsV2/com_apple_MobileAsset_DictionaryServices_dictionaryOSX/"):
    if entry.is_dir():
        try:
            infopath = os.path.join(entry.path,"Info.plist")
            with open(infopath) as infofile:
                if 'Dizionario italiano' in infofile.read():
                    dictpath = entry.path
                    break
        except Exception as e:
            pass

In [18]:
# Thanks to commenters for providing the base of this much nicer implementation!
# Save and run with $ python 0dedict.py
# You may need to hunt down the dictionary files yourself and change the awful path string below.
# This works for me on MacOS 10.14 Mohave

from struct import unpack
from zlib import decompress
import re
# filename = '/System/Library/Assets/com_apple_MobileAsset_DictionaryServices_dictionaryOSX/9f5862030e8f00af171924ebbc23ebfd6e91af78.asset/AssetData/Oxford Dictionary of English.dictionary/Contents/Resources/Body.data'
# filename = "/System/Library/AssetsV2/com_apple_MobileAsset_DictionaryServices_dictionaryOSX/bad4a5c5bf10a2dd135a07d7ad7b28e23d0aa7dd.asset/AssetData/Italian.dictionary/Contents/Resources/Body.data"
# filename = "/System/Library/AssetsV2/com_apple_MobileAsset_DictionaryServices_dictionaryOSX/a1fa9ccfe47b097a31a6b2ea94641e6e511855dd.asset/AssetData/Italian.dictionary/Contents/Resources/Body.data"

# Work Mac
filename = "/System/Library/AssetsV2/com_apple_MobileAsset_DictionaryServices_dictionaryOSX/d6b9a35f98ba22d746807da83ccbbaa6f83592c6.asset/AssetData/Italian.dictionary/Contents/Resources/Body.data"

# Home M1 Mac
filename = os.path.join(dictpath, "AssetData/Italian.dictionary/Contents/Resources/Body.data") #"/System/Library/AssetsV2/com_apple_MobileAsset_DictionaryServices_dictionaryOSX/f89a8ee38d37f4b519cf69009c2e071308b101f9.asset/AssetData/Italian.dictionary/Contents/Resources/Body.data"


f = open(filename, 'rb')

def gen_entry():
    f.seek(0x40)
    limit = 0x40 + unpack('i', f.read(4))[0]
    f.seek(0x60)
    while f.tell()<limit:
        sz, = unpack('i', f.read(4))
        buf = decompress(f.read(sz)[8:])

        pos = 0
        while pos < len(buf):
            chunksize, = unpack('i', buf[pos:pos+4])
            pos += 4

            entry = buf[pos:pos+chunksize]
            title = re.search('d:title="(.*?)"', entry.decode('utf-8')).group(1)
            yield title, entry

            pos += chunksize

wordlist = []
for word, definition in gen_entry():
    wordlist.append(word)

In [19]:
print(len(wordlist))

97926


# Parole italiane

```
git clone https://github.com/napolux/paroleitaliane.git

sudo apt-get install witalian

/usr/share/dict/italian
```

In [20]:
parole60  = "data/paroleitaliane/paroleitaliane/60000_parole_italiane.txt"
parole95  = "data/paroleitaliane/paroleitaliane/95000_parole_italiane_con_nomi_propri.txt"

parole280 = "data/paroleitaliane/paroleitaliane/280000_parole_italiane.txt"

paroleDebian = "data/italian"

# with open(paroleDebian, "r") as file:
#     words = [w.strip() for w in file.readlines() if "'" not in w]
    


In [21]:
words = [w for w in wordlist if len(w)>1 and '-' not in w and '(' not in w and ')' not in w and "'" not in w and "’" not in w]

In [22]:
words

['abaca',
 'abacà',
 'abaco',
 'abadessa',
 'abadia',
 'abalietà',
 'ab antiquo',
 'abarico',
 'abasia',
 'abate',
 'abatino',
 'abaton',
 'abaya',
 'abazia',
 'abaziale',
 'abbacare',
 'abbacchiamento',
 'abbacchiare',
 'abbacchiato',
 'abbacchiatura',
 'abbacchio',
 'abbacinamento',
 'abbacinante',
 'abbacinare',
 'abbaco',
 'abbadare',
 'abbadessa',
 'abbadia',
 'abbagliamento',
 'abbagliante',
 'abbagliare',
 'abbaglio',
 'abbaiare',
 'abbaiata',
 'abbaino',
 'abbaio',
 'abbaio',
 'abballare',
 'abballottare',
 'abbambinare',
 'abbambolato',
 'abbancare',
 'abbandonare',
 'abbandonato',
 'abbandonico',
 'abbandonismo',
 'abbandono',
 'abbarbagliamento',
 'abbarbagliare',
 'abbarbaglio',
 'abbarbaglio',
 'abbarbicamento',
 'abbarbicante',
 'abbarbicare',
 'abbarcare',
 'abbarcare',
 'abbarcatura',
 'abbarrare',
 'abbaruffare',
 'abbaruffio',
 'abbassabile',
 'abbassalingua',
 'abbassamento',
 'abbassare',
 'abbassata',
 'abbasso',
 'abbastanza',
 'abbate',
 'abbattere',
 'abbattibil

In [23]:
len(words)

96297

In [24]:
with open("data/paroleitaliane/paroleitaliane/1000_parole_italiane_comuni.txt", "r") as file:
    common_words = [w.strip() for w in file.readlines()]
    
common_words = [w for w in common_words if len(w)>2]
len(common_words)

1132

In [25]:
def create_semantle_data(all_words, solution_words, model, date, puzzle_number, verbose=False):
    # Day to use for the solution
    isoday = date.isoformat()
    
    # Solution
    # Seed the random number generator
    random.seed(a=isoday, version=2)
    # Pick the word
    query = random.choice(solution_words)
    if verbose:
        print("La soluzione per il {} è \"{}\"".format(isoday, query))
        
    # Distanze
    distances_list = [(word, cosine_distance(query, word, model)) for word in set(all_words+solution_words) if len(word)>1]
    
    # Distanze nell'ordine giusto
    distances_list_sorted = sorted(distances_list, key=lambda x: x[1],reverse=True)
    
    # Database per Semantle
    
    word_database = {
        _tuple[0]:
        {
            "s": _tuple[1],
            "r": _ix,
        } for _ix, _tuple in enumerate(distances_list_sorted)
    }
    
    # Le parole più vicine
    closest_words_list = [
        {
            "w": _tuple[0],
            "s": _tuple[1],
            "r": _ix,
        } for _ix, _tuple in enumerate(distances_list_sorted[:1000])
    ]
    
    solution_word = query
    
    # Statistiche
    
    nearest_word_similarity = closest_words_list[1]["s"]
    tenth_nearest_word_similarity = closest_words_list[9]["s"]
    thousandth_nearest_word_similarity = closest_words_list[999]["s"]

    day_stats = {
        "puzzle_number": puzzle_number,
        "nearest_word_similarity": nearest_word_similarity,
        "tenth_nearest_word_similarity": tenth_nearest_word_similarity,
        "thousandth_nearest_word_similarity": thousandth_nearest_word_similarity,
    }

    semantle_data = {
        "word_database": word_database,
        "closest_words_list": closest_words_list,
        "solution_word": solution_word,
        "day_stats": day_stats,
    }
    return semantle_data

In [26]:
wikidata = create_semantle_data(
    all_words=words,
    solution_words=common_words,
    model=wikimodel,
    date=datetime.date.today()-datetime.timedelta(days=2),
    puzzle_number=-1,
    verbose=True
)

ccdata = create_semantle_data(
    all_words=words,
    solution_words=common_words,
    model=ccmodel,
    date=datetime.date.today()-datetime.timedelta(days=2),
    puzzle_number=-1,
    verbose=True
)

La soluzione per il 2025-01-09 è "padrone"
La soluzione per il 2025-01-09 è "padrone"


In [27]:
wikidata = create_semantle_data(
    all_words=words,
    solution_words=common_words,
    model=wikimodel,
    date=datetime.date.today()-datetime.timedelta(days=1),
    puzzle_number=-1,
    verbose=True
)

La soluzione per il 2025-01-10 è "epoca"


In [28]:
# date=datetime.date.today()-datetime.timedelta(days=1)
# isoday = date.isoformat()
# print(isoday)
# random.seed(a=isoday, version=2)
# for i in range(10):
#     print(random.random())
# query = random.choice(common_words)
# print(query)

# date=datetime.date.today()-datetime.timedelta(days=0)
# isoday2 = date.isoformat()
# print(isoday2)
# random.seed(a=isoday2, version=2)
# for i in range(10):
#     print(random.random())
    
# # Pick the word
# query = random.choice(common_words)
# print(query)

In [29]:
# days_since_start = (datetime.date.today()-datetime.date(year=2022,month=9,day=9)).days
# print(len(common_words),days_since_start)

In [30]:
# ((len(common_words)-1)/len(common_words))**days_since_start

In [31]:
distances = [ccdata['word_database'][w]['s'] for w in ccdata['word_database']]
for d in distances:
    if np.isnan(d):
        print(d)

In [54]:
_old_query = ""
for days_ahead in range(365*10):
    _date = datetime.date(2022,9,9)+datetime.timedelta(days=days_ahead)

    # Day to use for the solution
    isoday = _date.isoformat()
    
    # Solution
    # Seed the random number generator
    random.seed(a=isoday, version=2)
    # Pick the word
    query = random.choice(common_words)
    if query == _old_query:
        print("On {} the word {} repeats!".format(_date, query))
    _old_query = query

On 2024-02-26 the word allora repeats!
On 2024-04-26 the word mattino repeats!
On 2027-08-14 the word assoluto repeats!
On 2032-05-15 the word affatto repeats!


In [55]:
print([w['w'] for w in wikidata['closest_words_list'][:50]])
# print(wikidata['word_database']['re'])

['espressione', 'espressività', 'locuzione', 'accezione', 'elocuzione', 'espressiva', 'intraducibilità', 'circonlocuzione', 'espressivo', 'traducibilità', 'letteralità', 'intraducibile', 'connotazione', 'esemplificazione', 'allusività', 'definitezza', 'espressionistico', 'esplicazione', 'interiezione', 'interlocuzione', 'parola', 'aggettivazione', 'letterale', 'espressionista', 'significante', 'estrinsecazione', 'inespressivo', 'icasticità', 'esprimere', 'allitterazione', 'pregnanza', 'denotazione', 'accentuazione', 'significare', 'espressionismo', 'significanza', 'definizione', 'determinatezza', 'idealizzazione', 'polirematica', 'negazione', 'sottinteso', 'assertività', 'pervasività', 'contrapposizione', 'antitesi', 'genericamente', 'enfatizzazione', 'significato', 'interiorizzazione']


In [56]:
print([w['w'] for w in ccdata['closest_words_list'][:50]])
# print(ccdata['word_database']['re'])

['causa', 'mancanza', 'conseguenza', 'concausa', 'dispetto', 'causare', 'colpa', 'assenza', 'carenza', 'scarsità', 'conseguente', 'perdita', 'imputabile', 'cagione', 'provocare', 'cognizione', 'mancato', 'nonostante', 'concomitanza', 'negligenza', 'sfortunatamente', 'malgrado', 'sconsideratezza', 'limitazione', 'ovviare', 'perorare', 'imputare', 'motivo', 'sfortuna', 'tale', 'terribile', 'defezione', 'subire', 'compensare', 'cessazione', 'indisposizione', 'consequenza', 'sopraggiungere', 'sopperire', 'ragione', 'dovuto', 'esclusione', 'presenza', 'ritardato', 'sopraggiunta', 'incomprensione', 'interruzione', 'rottura', 'determinare', 'disfunzione']


In [57]:
data = create_semantle_data(
    all_words=words,
    solution_words=common_words,
    model=wikimodel,
    date=datetime.date(2022,9,11),
    puzzle_number=1,
    verbose=True
)

La soluzione per il 2022-09-11 è "nobile"


In [58]:
data = create_semantle_data(
    all_words=words,
    solution_words=common_words,
    model=ccmodel,
    date=datetime.date(2022,9,11),
    puzzle_number=1,
    verbose=True
)
print(data["word_database"]["conte"])
print([n["w"] for n in data["closest_words_list"]])


La soluzione per il 2022-09-11 è "nobile"
{'s': 47.61, 'r': 21}
['nobile', 'nobiltà', 'nobiliare', 'aristocratico', 'nobildonna', 'nobiluomo', 'casato', 'illustre', 'patrizio', 'casata', 'aristocrazia', 'nobilitato', 'umile', 'nobilitare', 'gentiluomo', 'benestante', 'discendente', 'ignobile', 'famiglia', 'eminente', 'amabile', 'conte', 'principe', 'cortigiano', 'possidente', 'stirpe', 'plebeo', 'lignaggio', 'magnatizio', 'gentildonna', 'marchese', 'nobilitazione', 'cavaliere', 'barone', 'contessa', 'duca', 'rampollo', 'antico', 'notabile', 'decurionale', 'virile', 'facoltoso', 'rispettabile', 'prosapia', 'baronale', 'signorotto', 'marchesana', 'lodevole', 'ceto', 'vile', 'blasone', 'feudatario', 'fanciulla', 'giovane', 'regnante', 'schiatta', 'caritatevole', 'patriotta', 'prelato', 'rango', 'generoso', 'cinquecentesco', 'genealogista', 'regale', 'potente', 'decaduto', 'letterato', 'umanista', 'marchionale', 'temperante', 'rinascimentale', 'condottiere', 'filantropo', 'magnanimo', 'val

In [32]:
for days_ahead in range(365*5):
    _date = datetime.date(2022,9,9)+datetime.timedelta(days=days_ahead)
    if _date < datetime.date.today():
        continue
    # if _date < datetime.date(2025,9,6):
    #     continue
    data = create_semantle_data(
        all_words=words,
        solution_words=common_words,
        model=ccmodel,
        date=_date,
        puzzle_number=days_ahead,
        verbose=True
    )
    isopath = _date.isoformat()
    os.makedirs("AppleCC/"+isopath, exist_ok = True)
    with open("AppleCC/"+isopath+"/semantle.json", "w") as file:
        json.dump(data,file)
    with open("AppleCC/"+isopath+"/closest.json", "w") as file:
        json.dump(data['closest_words_list'],file)
    with open("AppleCC/"+isopath+"/stats.json", "w") as file:
        json.dump(data['day_stats'],file)
    with open("AppleCC/"+isopath+"/database.json", "w") as file:
        json.dump(data['word_database'],file)

La soluzione per il 2025-01-11 è "scrivere"
La soluzione per il 2025-01-12 è "guardia"
La soluzione per il 2025-01-13 è "notte"
La soluzione per il 2025-01-14 è "elevare"
La soluzione per il 2025-01-15 è "massimo"
La soluzione per il 2025-01-16 è "sedere"
La soluzione per il 2025-01-17 è "autore"
La soluzione per il 2025-01-18 è "grazia"
La soluzione per il 2025-01-19 è "molto"
La soluzione per il 2025-01-20 è "linea"
La soluzione per il 2025-01-21 è "massa"
La soluzione per il 2025-01-22 è "proposta"
La soluzione per il 2025-01-23 è "autore"
La soluzione per il 2025-01-24 è "campagna"
La soluzione per il 2025-01-25 è "necessità"
La soluzione per il 2025-01-26 è "tornare"
La soluzione per il 2025-01-27 è "attendere"
La soluzione per il 2025-01-28 è "sviluppo"
La soluzione per il 2025-01-29 è "sereno"
La soluzione per il 2025-01-30 è "sbagliare"
La soluzione per il 2025-01-31 è "posto"
La soluzione per il 2025-02-01 è "simile"
La soluzione per il 2025-02-02 è "scappare"
La soluzione per

In [ ]:
wikidata.keys()

In [59]:
_date

datetime.date(2032, 9, 5)

In [67]:
_date < datetime.date.today()

False

In [66]:
datetime.date.today()

datetime.date(2024, 9, 21)